In [1]:
# Import libraries
import pandas as pd
import numpy as np
import re
import string
import emoji

# ML libraries
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score


In [2]:
train_pos = pd.read_csv('/kaggle/input/arabic-sentiment-twitter-corpus/train_Arabic_tweets_positive_20190413.tsv', sep='\t', header=None, names=['tweet'])
train_neg = pd.read_csv('/kaggle/input/arabic-sentiment-twitter-corpus/train_Arabic_tweets_negative_20190413.tsv', sep='\t', header=None, names=['tweet'])
test_pos = pd.read_csv('/kaggle/input/arabic-sentiment-twitter-corpus/test_Arabic_tweets_positive_20190413.tsv', sep='\t', header=None, names=['tweet'])
test_neg = pd.read_csv('/kaggle/input/arabic-sentiment-twitter-corpus/test_Arabic_tweets_negative_20190413.tsv', sep='\t', header=None, names=['tweet'])

In [3]:
#Add sentiment labels
train_pos['sentiment'] = 'positive'
train_neg['sentiment'] = 'negative'
test_pos['sentiment'] = 'positive'
test_neg['sentiment'] = 'negative'
#Combine into single dataset
df = pd.concat([train_pos, train_neg, test_pos, test_neg], ignore_index=True)
print(df.head())
print(df['sentiment'].value_counts())

                                               tweet sentiment
0  نحن الذين يتحول كل ما نود أن نقوله إلى دعاء لل...  positive
1  وفي النهاية لن يبقىٰ معك آحدإلا من رأىٰ الجمال...  positive
2                                    من الخير نفسه 💛  positive
3  #زلزل_الملعب_نصرنا_بيلعب كن عالي الهمه ولا ترض...  positive
4  الشيء الوحيد الذي وصلوا فيه للعالمية هو : المس...  positive
sentiment
positive    28513
negative    28282
Name: count, dtype: int64


In [4]:
# Text Cleaning
def clean_text(text):
    text = str(text)
    text = text.lower()
    text = re.sub(r'http\S+|www\S+','', text)          # Remove URLs
    text = re.sub(r'@\w+','', text)                    # Remove mentions
    text = re.sub(r'#\w+','', text)                    # Remove hashtags
    text = re.sub(r'\d+','', text)                     # Remove numbers
    text = re.sub(r'[{}]'.format(string.punctuation), '', text)  # Remove punctuation
    text = re.sub(r'[ًٌٍَُِّْـ]', '', text)           # Remove Arabic diacritics
    text = emoji.replace_emoji(text, replace='')       # Remove emojis
    text = re.sub(r'\s+', ' ', text).strip()          # Remove extra spaces
    return text

df['clean_text'] = df['tweet'].apply(clean_text)

In [5]:
# Feature Extraction 
#TF-IDF
use_tfidf = False  # Set True for TF-IDF, False for CountVectorizer + N-grams

if use_tfidf:
    vectorizer = TfidfVectorizer(ngram_range=(1,3), max_features=5000)
else:
    vectorizer = CountVectorizer(ngram_range=(1,3), max_features=5000)

X = vectorizer.fit_transform(df['clean_text'])
y = df['sentiment']

In [6]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [7]:
# ML Models Training & Evaluation
models = {
    'Logistic Regression': LogisticRegression(max_iter=1000),
    'Naive Bayes': MultinomialNB(),
    'Random Forest': RandomForestClassifier(n_estimators=100),
    'SVM': SVC(kernel='linear')
}

results = []

for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')
    results.append({'Model': name, 'Accuracy': acc, 'F1-Score': f1})


In [8]:
#Results Table
results_df = pd.DataFrame(results)
print(results_df)


                 Model  Accuracy  F1-Score
0  Logistic Regression  0.738885  0.737963
1          Naive Bayes  0.707369  0.707355
2        Random Forest  0.777533  0.777239
3                  SVM  0.736244  0.733936
